    Run `python make_pooled_datasets.py configs/3T_NIST_T1maps.json 3T_NIST_T1maps` first

In [1]:
from pathlib import Path
import pandas as pd
import json

In [2]:
configFile = Path('configs/3T_NIST_T1maps.json')

In [3]:
with open(configFile) as json_file:
    configJson = json.load(json_file)

In [4]:
def create_database(configJson):
    columns = [
    'id',
    'OSF dataset', 'OSF link', 'NIFTI filename', 'Data type',
    'contact', 'email',
    'site name', 'MRI vendor', 'MRI version', 'MRI field',
    'sample type', 'phantom version', 'phantom serial number', 'phantom temperature',
    'sequence name', 'sequence type', 'matrix size', 'resolution', 'dimension', 'TR', 'echo time', 'TI', 'bandwidth'
    ]
    
    df = pd.DataFrame(columns=columns)
    df = df.set_index('id')
    
    df = parse_dataset_json(df, configJson)
    
    return df

In [5]:
def parse_dataset_json(df, configJson):

    id = 1
    subid = 1

    for dataset_name in configJson:
        db_id = id+subid*0.001
        for key1 in configJson[dataset_name]:
            if key1 == 'datasets':
                for key2 in configJson[dataset_name][key1]:
                    db_id = id+subid*0.001
                    subid = subid+1
                    dataset_series = {
                        'OSF dataset': dataset_name,
                        'OSF link': configJson[dataset_name]['OSF_link'],
                        'NIFTI filename': configJson[dataset_name]['datasets'][key2]['imagePath'],
                        'Data type': configJson[dataset_name]['datasets'][key2]['dataType']
                    }
                    
                    t1File = configJson[dataset_name]['datasets'][key2]['imagePath']
                    t1JsonFile = '3T_NIST_T1maps' / Path(t1File[:-7] + '.json')
                    
                    with open(t1JsonFile) as json_file:
                        t1Json = json.load(json_file)
                    

                    
                    databaset_series = parse_t1_json(dataset_series, t1Json)
                    df = df.append(pd.Series(dataset_series, index = df.columns, name = db_id))
        # Increment dataset ID counter
        id = id + 1
    
        # Reset subdataset ID counter
        subid = 1
    return df

In [6]:
def parse_t1_json(databaset_series, t1Json):
    databaset_series.update({
        'contact': t1Json['submitter']['contact'],
        'email': t1Json['submitter']['email'],
    })

    databaset_series.update({
        'site name': t1Json['site']['name'],
        'MRI vendor': t1Json['site']['manufacturer'],
        'MRI version': t1Json['site']['version'],
        'MRI field': t1Json['site']['field'],
    })
    
    if 'temperature' in t1Json['sample']:
        temp = t1Json['sample']['temperature']
    else:
        temp = None
    
    databaset_series.update({
        'sample type': t1Json['sample']['type'],
        'phantom version': t1Json['sample']['version'],
        'phantom serial number': t1Json['sample']['serial_number'],
        'phantom temperature': temp,
    })

    if 'bandwidth' in t1Json['sequence']:
        bandwidth = t1Json['sequence']['bandwidth']
    else:
        bandwidth = None

    databaset_series.update({
        'sequence name': t1Json['sequence']['name'],
        'sequence type': t1Json['sequence']['type'],
        'matrix size': t1Json['sequence']['matrix_size'],
        'resolution': t1Json['sequence']['resolution'],
        'dimension': t1Json['sequence']['dimension'],
        'TR': t1Json['sequence']['repetition_time'],
        'echo time': t1Json['sequence']['echo_time'],
        'TI': t1Json['sequence']['inversion_times'],    
        'bandwidth': bandwidth,        
    })
    return databaset_series

In [7]:
df = create_database(configJson)

In [8]:
df

,OSF dataset,OSF link,NIFTI filename,Data type,contact,email,site name,MRI vendor,MRI version,MRI field,...,phantom temperature,sequence name,sequence type,matrix size,resolution,dimension,TR,echo time,TI,bandwidth
id,,,,,,,,,,,,,,,,,,,,,
1.001,matthewgrechsollars_ICL_NIST,https://osf.io/7k96d/download/,20200121_matthewgrechsollars_ICL_NIST/20200121...,Magnitude,Matthew Grech-Sollars,m.grech-sollars@imperial.ac.uk,Imperial College London,Siemens,Prisma VE11C,3.00,...,None,se_TI50/400/1100/2400,Spin-echo inversion recovery,[256 256 1],[1 1 2],2D,2550.0,14.0,"[50, 400, 1100, 2400]",130
1.002,matthewgrechsollars_ICL_NIST,https://osf.io/7k96d/download/,20200121_matthewgrechsollars_ICL_NIST/20200121...,Complex,Matthew Grech-Sollars,m.grech-sollars@imperial.ac.uk,Imperial College London,Siemens,Prisma VE11C,3.00,...,None,se_TI50/400/1100/2400,Spin-echo inversion recovery,[256 256 1],[1 1 2],2D,2550.0,14.0,"[50, 400, 1100, 2400]",130
2.001,siyuanhu_casewestern_NIST,https://osf.io/46nd9/download/,20200124_siyuanhu_casewestern_NIST/20200124_si...,Magnitude,Siyuan Hu,siyuan.hu@case.edu,"Biomedical Engineering, Case Western Reserve U...",Siemens,Skyra,3.00,...,19.1,SE_IR,Spin-echo inversion recovery,[256 256 1],[1 1 2],2D,2550.0,14.0,"[50, 400, 1100, 2500]",None
3.001,iveslevesque_muhc_mgh_NIST,https://osf.io/nk6ae/download/,20200203_iveslevesque_muhc_mgh_NIST/20200203_i...,Magnitude,Ives Levesque,ives.levesque@mcgill.ca,McGill University Health Centre - Montreal Gen...,Siemens,syngo MR E11,3.00,...,None,T1_se_cor_TI,Spin-echo inversion recovery,[256 256 1],[1 1 2],2D,2550.0,14.0,"[50, 400, 1100, 2500]",390
4.001,mrel_usc_NIST,https://osf.io/t5y28/download/,20200204_mrel_usc_NIST/20200204_mrel_usc_GE3T_...,Magnitude,Nam Lee,namgyunl@usc.edu,Keck Medical Center of University of Southern ...,GE,Signa HDxt,3.00,...,20.9,"2D Spin Echo, IR",Spin-echo inversion recovery,[256 256 1],[1 1 2],2D,2550.0,14.0,"[50, 400, 1100, 2500]",244.141
4.002,mrel_usc_NIST,https://osf.io/t5y28/download/,20200204_mrel_usc_NIST/20200204_mrel_usc_GE3T_...,Magnitude,Nam Lee,namgyunl@usc.edu,Keck Medical Center of University of Southern ...,GE,Signa HDxt,3.00,...,20.9,"2D Spin Echo, IR",Spin-echo inversion recovery,[256 256 1],[1 1 2],2D,2550.0,14.0,"[50, 400, 1100, 2500]",244.141
4.003,mrel_usc_NIST,https://osf.io/t5y28/download/,20200204_mrel_usc_NIST/20200209_mrel_usc_GE3T_...,Magnitude,Nam Lee,namgyunl@usc.edu,Keck Medical Center of University of Southern ...,GE,Signa HDxt,3.00,...,20.9,"2D Spin Echo, IR",Spin-echo inversion recovery,[256 256 1],[1 1 2],2D,2550.0,14.0,"[50, 400, 1100, 2500]",244.141
4.004,mrel_usc_NIST,https://osf.io/t5y28/download/,20200204_mrel_usc_NIST/20200209_mrel_usc_GE3T_...,Magnitude,Nam Lee,namgyunl@usc.edu,Keck Medical Center of University of Southern ...,GE,Signa HDxt,3.00,...,20.9,"2D Spin Echo, IR",Spin-echo inversion recovery,[256 256 1],[1 1 2],2D,10000.0,14.0,"[50, 400, 1100, 2500]",244.141
4.005,mrel_usc_NIST,https://osf.io/t5y28/download/,20200204_mrel_usc_NIST/20200209_mrel_usc_GE3T_...,Magnitude,Nam Lee,namgyunl@usc.edu,Keck Medical Center of University of Southern ...,GE,Signa HDxt,3.00,...,20.9,"2D Spin Echo, IR",Spin-echo inversion recovery,[256 256 1],[1 1 2],2D,2550.0,14.0,"[50, 400, 1100, 2500]",244.141


In [10]:
df.to_excel("database.xlsx")  